# Forecasting Bonus-Driven Conversion Uplift

This notebook examines how a limited-time bonus campaign impacts conversion rates and whether the uplift is sustainable. We model the observed data with exponential smoothing to estimate performance over the next two weeks.

## 1. Load campaign conversion data
The dataset contains 60 days of synthetic conversion rates with a flag marking the period after the bonus launch.

In [1]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import matplotlib.pyplot as plt
from pathlib import Path


In [2]:
data_path = Path('..') / 'data' / 'synthetic_conversion_data.csv'
df = pd.read_csv(data_path, parse_dates=['date'])
df = df.sort_values('date').reset_index(drop=True)
df.head()

,date,channel,registrations,first_deposits,bonus_claims,deposit_amount
0,2025-08-01,SEO,520,210,0,16800
1,2025-08-01,Affiliates,460,190,95,15400
2,2025-08-01,PaidSearch,380,110,45,9700
3,2025-08-01,Email,240,85,60,8100
4,2025-08-02,SEO,510,208,0,16750


## 2. Fit exponential smoothing model
We apply single exponential smoothing to capture the level shift from the bonus and forecast the next 14 days.

In [3]:
model = ExponentialSmoothing(df['conversion_rate'], trend=None, seasonal=None, initialization_method='heuristic')
fit = model.fit(smoothing_level=0.35, optimized=False)
forecast_horizon = 14
forecast_index = pd.date_range(df['date'].iloc[-1] + pd.Timedelta(days=1), periods=forecast_horizon, freq='D')
forecast_series = pd.Series(fit.forecast(forecast_horizon), index=forecast_index)
forecast_series.head()

KeyError: 'conversion_rate'

## 3. Visualize historical and forecasted conversion
The chart highlights the campaign launch date and compares actual conversions with the projected trajectory.

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df['date'], df['conversion_rate'], label='Actual', color='#1E66C5')
plt.plot(forecast_series.index, forecast_series.values, label='Forecast', color='#F2932D', linestyle='--')
campaign_start = df.loc[~df['is_post_campaign'].shift(fill_value=False) & df['is_post_campaign'], 'date'].min()
if pd.notnull(campaign_start):
    plt.axvline(campaign_start, color='#C81E2E', linestyle=':', linewidth=2, label='Campaign launch')
plt.title('Bonus Campaign Conversion Forecast')
plt.ylabel('Conversion rate')
plt.xlabel('Date')
plt.legend()
plt.grid(True, alpha=0.3)
output_path = Path('..') / 'reports' / 'conversion_forecast_chart.png'
plt.tight_layout()
plt.savefig(output_path, dpi=150)
plt.show()


## 4. Insight
The exponential smoothing forecast suggests that the bonus-induced uplift begins to taper off but remains above the pre-campaign baseline during the forecast window. Marketing can use this insight to plan follow-on promotions or retention offers if additional lift is required.